In [1]:
# This notebook is about self-written peak to peak detection.
# MNE has own peak to peak detection in annotate_amplitude, but it used different settings and the process 
# of calculation there is not yet clear. We may use their way or may keep this one.


#Load data, filter, make folders
import numpy as np
import plotly.graph_objects as go
import pandas as pd

from main_meg_qc import initial_stuff

In [2]:
duration=60 #1 min
n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration)

Opening raw data file ../data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/../data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif :

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:17: RuntimeWarning: This filename (../data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(data_file)


    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 4943000 ... 5198999 =   4943.000 ...  5198.999 secs
Ready.
Reading 0 ... 60000  =      0.000 ...    60.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 100.00 Hz: -6.02, -6.02 dB

Trigger channel has a non-zero initial value of 18 (consider using

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:84: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  raw_bandpass_resamp.resample(sfreq=h_freq*5)


Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
42 events found
Event IDs: [    9    19    20    21    22    23    27    31 16393 16402 16403 16405
 16411 32741 32749 32750 32759]
Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
5 events found
Event IDs: [ 9 19 20 21]
Not setting metadata
5 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Using data from preloaded Raw for 5 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
5 matching events found
No baseline correction applied
8 projection items activated
Using data from preloaded Raw for 5 events and 1201 original time points ...
0 bad epochs dropped


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:84: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  raw_bandpass_resamp.resample(sfreq=h_freq*5)
/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:112: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(data, stim_channel=stim_channel, min_duration=event_dur)


In [3]:
# Separate data for mags and grads in 2 arrays.

selected_mags = [item[1] for item in mags]
selected_grads = [item[1] for item in grads]
data_mags, _ = filtered_d_resamp[selected_mags, :]  
data_grads, _ = filtered_d_resamp[selected_grads, :]  

sfreq = filtered_d_resamp.info['sfreq']

In [4]:
#EXAMPLE: find peaks of the first magnetometer channel:

from mne.preprocessing import peak_finder

thresh=(max(data_mags[0]) - min(data_mags[0])) / 1.7 #THRESHOLD FOR IDENTIFICATION ADJUST HERE
# The amount above surrounding data for a peak to be identified. Larger values mean the algorithm 
# is more selective in finding peaks. If None, use the default of (max(x0) - min(x0)) / 4.

pos_peak_locs, pos_peak_magnitudes = peak_finder(data_mags[0], extrema=1, thresh=thresh) #positive peaks
neg_peak_locs, neg_peak_magnitudes = peak_finder(data_mags[0], extrema=-1, thresh=thresh) #negative peaks


t=np.arange(0, duration, 1/sfreq) 
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=data_mags[0], name='data'));
fig.add_trace(go.Scatter(x=t[pos_peak_locs], y=pos_peak_magnitudes, mode='markers', name='+peak'));
fig.add_trace(go.Scatter(x=t[neg_peak_locs], y=neg_peak_magnitudes, mode='markers', name='-peak'));
fig.update_layout(
    title={
    'text': "First magnetometer: peaks detected",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title="Time in seconds",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'))
    
fig.show()


#https://www.codegrepper.com/code-examples/python/go.Scatter+no+line

Found 13 significant peaks
Found 12 significant peaks


In [5]:
#EXAMPLE part2: Now find pairs of peaks (pair of neg+positive) and calculate mean over these pairs:

pair_dist_sec=0.5 #max distance between neg and pos peak to count them as pair, in sec
pair_dist=pair_dist_sec*sfreq
print(pair_dist)

pairs_magnitudes=[]
pairs_locs=[]

for posit_pair_ind, posit_pair in enumerate(pos_peak_locs):
    #find all negative peaks which lay at the set distance from positive ones
    neg_pair_ind=np.where(np.logical_and(neg_peak_locs>=posit_pair-pair_dist/2, neg_peak_locs<=posit_pair+pair_dist/2))
    if neg_pair_ind[0].size != 0: #if there are negative peaks - save pos+negative into pairs
        pairs_locs.append([pos_peak_locs[posit_pair_ind], neg_peak_locs[neg_pair_ind[0][0]]])
        pairs_magnitudes.append([pos_peak_magnitudes[posit_pair_ind], neg_peak_magnitudes[neg_pair_ind[0][0]]])
    #else:
        #print ('No pair detected')

amplitude=np.zeros(len(pairs_magnitudes),)
for i, pair in enumerate(pairs_magnitudes):
    print(pair)
    amplitude[i]=pair[0]-pair[1]

mean_amplitude=np.mean(amplitude)


#This for plotting only:
list_pairs_locs = [i for sub in pairs_locs for i in sub]
list_pairs_magnitudes = [i for sub in pairs_magnitudes for i in sub]


fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=data_mags[0], name='data'));
fig.add_trace(go.Scatter(x=t[list_pairs_locs], y=list_pairs_magnitudes, mode='markers', name='pair'));
fig.update_layout(
    title={
    'text': "First magnetometer: PAIRS of peaks detected",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title="Time in seconds",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'))
    
fig.show()

print(amplitude)
print('mean: ', mean_amplitude)


250.0
[1.2750223727985316e-12, -2.348914740882849e-12]
[1.6536776681718568e-12, -2.7007423152539433e-12]
[1.843137032278871e-12, -2.2134551974137203e-12]


[3.62393711e-12 4.35441998e-12 4.05659223e-12]
mean:  4.011649775599924e-12


In [6]:
def neighbour_peak_amplitude(pair_dist_sec: float, sfreq: int, pos_peak_locs:np.ndarray, neg_peak_locs:np.ndarray, pos_peak_magnitudes: np.ndarray, neg_peak_magnitudes: np.ndarray) -> float:

    ''' Function finds a pair: postive+negative peak and calculates the amplitude between them. 
    If no neighbour is found withing given distance - this peak is skipped. 
    If several neighbours are found - several pairs are created. 
    As the result a mean peak-to-peak distance is calculated over all detected pairs for given chunck of data
    
    Args:
    pair_dist_sec (float): maximum distance in seconds which is allowed for negative+positive peaks to be detected as a pair 
    sfreq: sampling frequency of data. Attention to which data is used! original or resampled.
    pos_peak_locs (np.ndarray): output of peak_finder (Python) function - positions of detected Positive peaks
    neg_peak_locs (np.ndarray): output of peak_finder (Python) function - positions of detected Negative peaks
    pos_peak_magnitudes (np.ndarray): output of peak_finder (Python) function - magnitudes of detected Positive peaks
    neg_peak_magnitudes (np.ndarray): output of peak_finder (Python) function - magnitudes of detected Negative peaks

    Returns:
    (float): mean value over all detected peak pairs for this chunck of data.
    '''

    pair_dist=pair_dist_sec*sfreq
    pairs_magnitudes=[]
    pairs_locs=[]

    for posit_pair_ind, posit_pair in enumerate(pos_peak_locs):
        neg_pair_ind=np.where(np.logical_and(neg_peak_locs>=posit_pair-pair_dist/2, neg_peak_locs<=posit_pair+pair_dist/2))
        if neg_pair_ind[0].size != 0:
            pairs_locs.append([pos_peak_locs[posit_pair_ind], neg_peak_locs[neg_pair_ind[0][0]]])
            pairs_magnitudes.append([pos_peak_magnitudes[posit_pair_ind], neg_peak_magnitudes[neg_pair_ind[0][0]]])

    if len(pairs_magnitudes)==0:
        return np.nan

    amplitude=np.zeros(len(pairs_magnitudes),)
    for i, pair in enumerate(pairs_magnitudes):
        amplitude[i]=pair[0]-pair[1]

    return np.mean(amplitude)

In [7]:
def peak_amplitude_per_epoch(mg_names: list, df_epoch_mg: pd.DataFrame, sfreq: int, n_events: int, thresh_lvl: float, pair_dist_sec: float) -> pd.DataFrame:

    '''Function calculates peak-to-peak amplitude for every epoch and every channel (mag or grad).

    Args:
    mg_names (list of tuples): channel name + its index
    df_epoch_mg (pd. Dataframe): data frame containing data for all epochs for mags  or grads
    sfreq: sampling frequency of data. Attention to which data is used! original or resampled.
    n_events (int): number of events in this peace of data
    thresh_lvl (float): defines how high or low need to peak to be to be detected, this can also be changed into a sogle value later
        used in: max(data_ch_epoch) - min(data_ch_epoch)) / thresh_lvl 
    pair_dist_sec (float): maximum distance in seconds which is allowed for negative+positive peaks to be detected as a pair 

    Returns:
    df_pp_ampl_mg (pd.DataFrame): contains the mean peak-to-peak aplitude for each epoch for each channel

    '''
    
    dict_mg = {}

    mg_only_names=[n[0] for n in mg_names]

    epoch_numbers=[ep for ep in range(n_events)]
    for ep in epoch_numbers: #loop over each epoch
        rows_for_ep = [row for row in df_epoch_mg.iloc if row.epoch == ep] #take all rows of 1 epoch, all channels.
  
        peak_ampl_epoch=[]
        for ch_name in mg_only_names: #loop over channel names
            data_ch_epoch = [row_mg[ch_name] for row_mg in rows_for_ep] #take the data for 1 epoch for 1 channel

            thresh=(max(data_ch_epoch) - min(data_ch_epoch)) / thresh_lvl 
            #can also change the whole thresh to a single number setting

            pos_peak_locs, pos_peak_magnitudes = peak_finder(data_ch_epoch, extrema=1, thresh=thresh, verbose=False) #positive peaks
            neg_peak_locs, neg_peak_magnitudes = peak_finder(data_ch_epoch, extrema=-1, thresh=thresh, verbose=False) #negative peaks
            
            pp_ampl=neighbour_peak_amplitude(pair_dist_sec, sfreq, pos_peak_locs, neg_peak_locs, pos_peak_magnitudes, neg_peak_magnitudes)
            peak_ampl_epoch.append(pp_ampl)

        dict_mg[ep] = peak_ampl_epoch

    df_pp_ampl_mg = pd.DataFrame(dict_mg, index=mg_only_names)

    return df_pp_ampl_mg 

In [8]:
df_pp_ampl_mags=peak_amplitude_per_epoch(mg_names=mags, df_epoch_mg=df_epochs_mags, sfreq=sfreq, n_events=n_events, thresh_lvl=2, pair_dist_sec=0.5)

df_pp_ampl_grads=peak_amplitude_per_epoch(mg_names=grads, df_epoch_mg=df_epochs_grads, sfreq=sfreq, n_events=n_events, thresh_lvl=2, pair_dist_sec=0.5)


In [9]:
df_pp_ampl_mags

,0,1,2,3,4
MEG0111,5.719000e-13,5.027378e-13,4.829851e-13,6.140669e-13,4.872578e-13
MEG0121,2.833230e-13,4.584307e-13,3.012911e-13,6.155757e-13,3.127960e-13
MEG0131,2.999946e-13,3.071649e-13,2.780427e-13,3.505080e-13,3.104385e-13
MEG0141,7.033084e-13,5.715650e-13,4.849414e-13,6.460403e-13,4.393480e-13
MEG0211,4.342805e-13,4.114248e-13,4.298878e-13,3.505291e-13,4.766336e-13
...,...,...,...,...,...
MEG2541,6.452759e-13,6.431559e-13,6.826575e-13,7.530265e-13,6.339250e-13
MEG2611,5.494271e-13,NaN,3.934769e-13,4.746543e-13,4.611221e-13
MEG2621,8.620134e-13,7.965889e-13,6.088931e-13,8.371624e-13,NaN
MEG2631,5.603891e-13,6.331254e-13,5.489905e-13,7.982894e-13,7.183147e-13


In [10]:
#plot using new universal plotting function:

from universal_plots import boxplot_channel_epoch_hovering_plotly

fig_m_pp_ampl_epoch, fig_path_m_pp_ampl_epoch=boxplot_channel_epoch_hovering_plotly(df_mg=df_pp_ampl_mags, ch_type='Magnetometers', sid='1', what_data='peaks')
fig_g_pp_ampl_epoch, fig_path_g_pp_ampl_epoch=boxplot_channel_epoch_hovering_plotly(df_mg=df_pp_ampl_grads, ch_type='Gradiometers', sid='1', what_data='peaks')

fig_m_pp_ampl_epoch.show()
fig_g_pp_ampl_epoch.show()

In [ ]:
from universal_html_report import make_peak_html_report

list_of_figure_paths=[fig_path_m_pp_ampl_epoch, fig_path_g_pp_ampl_epoch]

make_peak_html_report(sid='1', what_data='peaks', list_of_figure_paths=list_of_figure_paths)